Populate MYSQL Weather database with synthetic static data and with historical weather data extracted from open-meteo API

In [3]:
# imports including classes from the create script

from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Date, Float, Enum, Numeric, ForeignKey
from sqlalchemy.dialects.mysql import LONGTEXT
from sqlalchemy import MetaData
from sqlalchemy import Index

from sqlalchemy_utils import database_exists, create_database, drop_database

from sqlalchemy.orm import Session
from sqlalchemy.orm import sessionmaker

import pymysql

import pandas as pd
import datetime as datetime

from create_mysql_db_weather import (
    engine,
    Base,
    Location,
    Weather_Data,
    Customer,
    Device_Component,
    Device
)


In [6]:
# establish the session with database engine

Session = sessionmaker(bind=engine)
session = Session()

In [7]:
# create populate_table function that creates rec object 
# with csv files mapping to the DB tables and columns that we want to populate


def populate_table(csv_file,table):

    df = pd.read_csv(csv_file, sep = ',', encoding = 'latin-1')
    df = df.dropna()
    #i=0
    
    for i, val in enumerate(df.values):
        if table == 'Location':
            rec = Location (
                location_id = val[0],
                latitude = val[1],
                longitude = val[2],
                city = val[3],
                region = val[4],
                country = val[5]

                
            )
        
        elif table == 'Weather_Data':
            rec = Weather_Data (

                location_id = val[1],
                time = val[2],
                shortwave_radiation_sum_MJ_m2 = val[3],
                temperature_2m_max_C = val[4],
                precipitation_sum_mm = val[5],
                relative_humidity_2m_mean_percent = val[6],
                temperature_2m_mean_C = val[7],
                temperature_2m_min_C = val[8]     

            )
        elif table == 'Customer':
            rec = Customer (
                customer_id = val[0],
                customer_name = val[1],
                location_id = val[2],
                safety_risk = val[3],
                difficult_access = val[4]

            )  

        elif table == 'Device_Component':
            rec = Device_Component (
                component_id = val[0],
                manufacturer = val[1],
                category = val[2],
                device_type = val[3],
                risk_type = val[4]

            )             
        else:
            rec = Device (
                device_id = val[0],
                component_1 = val[1],
                component_2 = val[2],
                component_3 = val[3],
                location_id = val[4],

            )
            
        session.add(rec)
        
    session.commit()


Insert data from CSV files into MYSQL table in strict order starting with Location table that has no dependencies and continuing with tables that depend on it

In [8]:
# insert the data from CSV files to the Location table in database


populate_table('synth_data/location.csv','Location')


In [10]:
populate_table('synth_data/customer.csv','Customer')

In [11]:
populate_table('synth_data/device_component.csv','Device_Component')

In [ ]:
populate_table('synth_data/device_component.csv','Device_Component')

In [ ]:
populate_table('meteo_data/historical_weather_load.csv', 'Weather_Data')